In [1]:
# Diagnostics: run this after selecting the kernel "Python (dental-xray)"
import sys, platform
print("Python:", sys.version)
print("Executable:", sys.executable)
print("Platform:", platform.platform())
try:
    from ultralytics import YOLO
    import torch, numpy as np
    print("ultralytics, torch, numpy OK")
    print("Torch:", torch.__version__)
    print("Device:", 'MPS' if torch.backends.mps.is_available() else 'CUDA' if torch.cuda.is_available() else 'CPU')
except Exception as e:
    print("Import error:", e)

Python: 3.13.5 | packaged by Anaconda, Inc. | (main, Jun 12 2025, 11:23:37) [Clang 14.0.6 ]
Executable: /Users/theatulgupta/Desktop/Deep Learning Project/dental-xray-cavity-detection/.venv/bin/python
Platform: macOS-26.0.1-arm64-arm-64bit-Mach-O
ultralytics, torch, numpy OK
Torch: 2.9.0
Device: MPS
ultralytics, torch, numpy OK
Torch: 2.9.0
Device: MPS


# YOLOv12 Training for Dental X-ray Cavity Detection

## About YOLOv12

YOLOv12 is an experimental/community variant of YOLO that may offer:

- Different architecture optimizations
- Potentially faster inference
- Different accuracy/speed tradeoffs

**Note**: YOLOv12 might be available through custom implementations or specific Ultralytics versions.

## This Notebook

We'll train YOLOv12 on the same dental X-ray dataset and compare it with YOLOv8:

1. Use the same dataset prepared in `01_prepare_dataset.ipynb`
2. Train with same configuration for fair comparison
3. Save weights to `models/yolov12_best.pt`
4. Compare inference speed and accuracy with YOLOv8


In [2]:
# Install required dependencies (uncomment if running for the first time)
# !pip install -q ultralytics==8.2.0 opencv-python pillow matplotlib numpy torch torchvision

In [3]:
# ============================================================
# SETUP PATHS AND IMPORTS
# ============================================================

from ultralytics import YOLO
from pathlib import Path
import shutil

# Detect project root (works from notebooks/ or project root)
project_root = (Path('..').resolve() if Path.cwd().name == 'notebooks' else Path('.').resolve())

# Define paths
processed = project_root / 'data' / 'processed'
dataset_yaml = processed / 'dataset.yaml'
models_dir = project_root / 'models'
models_dir.mkdir(parents=True, exist_ok=True)

# Verify dataset.yaml exists
if not dataset_yaml.exists():
    raise FileNotFoundError(
        f"dataset.yaml not found at {dataset_yaml}\n"
        "Please run '01_prepare_dataset.ipynb' first!"
    )

print('✓ Project root:', project_root)
print('✓ Dataset config:', dataset_yaml)
print('✓ Models directory:', models_dir)

✓ Project root: /Users/theatulgupta/Desktop/Deep Learning Project/dental-xray-cavity-detection
✓ Dataset config: /Users/theatulgupta/Desktop/Deep Learning Project/dental-xray-cavity-detection/data/processed/dataset.yaml
✓ Models directory: /Users/theatulgupta/Desktop/Deep Learning Project/dental-xray-cavity-detection/models


## Step 1: PyTorch Compatibility Patch

Fix for PyTorch 2.6+ to allow loading pretrained weights.


In [ ]:
# PyTorch 2.6+ compatibility patch
import torch
import warnings
import os

# Suppress warnings
warnings.filterwarnings('ignore')
os.environ['PYTHONWARNINGS'] = 'ignore'

_ORIGINAL_TORCH_LOAD = torch.load

def _patched_load(*args, **kwargs):
    if 'weights_only' not in kwargs:
        kwargs['weights_only'] = False
    return _ORIGINAL_TORCH_LOAD(*args, **kwargs)

torch.load = _patched_load
print("✓ PyTorch patch applied")

✓ PyTorch compatibility patch applied successfully
  - torch.load() will use weights_only=False by default
  - This fixes Ultralytics model loading errors on PyTorch 2.6+



## Step 2: Load YOLO Model

Load YOLOv11/v12 model (tries multiple versions).


In [ ]:
# Load YOLO model (try v11, v12, or v9)
models_to_try = ['yolo11n.pt', 'yolov11n.pt', 'yolo12n.pt', 'yolov12n.pt', 'yolo9n.pt']
model = None
model_name = None

for m in models_to_try:
    try:
        model = YOLO(m, verbose=False)
        model_name = m
        print(f"✓ Loaded {m}")
        break
    except:
        continue

if model is None:
    raise RuntimeError("No YOLO model available. Try: pip install --upgrade ultralytics")

Loading yolo12n.pt pretrained model...
✓ Model loaded successfully
  - Model type: YOLOv12n
  - Using transfer learning from COCO dataset


## Step 3: Training Configuration

Set hyperparameters (same as YOLOv8 for fair comparison).

**Note:** Ultralytics will show its standard training output including:

- Epoch progress with box_loss, cls_loss, dfl_loss metrics
- Validation results after each epoch
- This is normal and shows training is working correctly


In [ ]:
# Training configuration (same as YOLOv8 for fair comparison)
train_config = {
    'data': str(dataset_yaml),
    'epochs': 50,
    'imgsz': 640,
    'device': 'mps',
    'verbose': True,  # Keep verbose for progress bar
    'name': 'yolov12_dental',
    'patience': 10,
    'save': True,
    'plots': True,
}

print(f"Training for {train_config['epochs']} epochs on {train_config['device']}")

Training Configuration:
  data           : /Users/theatulgupta/Desktop/Deep Learning Project/dental-xray-cavity-detection/data/processed/dataset.yaml
  epochs         : 1
  imgsz          : 640
  device         : mps
  verbose        : True
  name           : yolov12_dental
  patience       : 10
  save           : True
  plots          : True


## Step 4: Train Model

Train YOLO model on dental X-ray dataset.


In [ ]:
# Train the model
print("Starting training...")
results = model.train(**train_config)
print("Training complete!")


✓ Ultralytics logger configured to hide verbose startup messages.


## Step 5: Save Model

Copy best weights to models directory.


In [ ]:
# Save best model weights
try:
    latest_run_dir = Path(results.save_dir)
    source_weights = latest_run_dir / 'weights' / 'best.pt'
    dest_weights = models_dir / 'yolov12_best.pt'

    if source_weights.exists():
        shutil.copy(source_weights, dest_weights)
        size_mb = dest_weights.stat().st_size / (1024*1024)
        print(f"✓ Model saved: {dest_weights} ({size_mb:.1f} MB)")
    else:
        print(f"✗ Weights not found at {source_weights}")
        
except NameError:
    print("✗ Run training cell first")



STARTING YOLOV12 TRAINING
Dataset: /Users/theatulgupta/Desktop/Deep Learning Project/dental-xray-cavity-detection/data/processed/dataset.yaml
Epochs: 1
Image Size: 640
Device: mps



## Step 6: Test Inference

Test the trained model on a sample image.


In [ ]:
# Test inference on sample image
from PIL import Image
import matplotlib.pyplot as plt
import cv2

trained_model = YOLO(str(dest_weights))

val_images_dir = processed / 'val' / 'images'
test_images = list(val_images_dir.glob('*.png')) + list(val_images_dir.glob('*.jpg'))

if test_images:
    test_img = test_images[0]
    results = trained_model.predict(source=str(test_img), verbose=False)
    
    print(f"Testing on: {test_img.name}")
    print(f"Detected {len(results[0].boxes)} cavities")
    
    # Visualize
    annotated_img = results[0].plot()
    annotated_img_rgb = cv2.cvtColor(annotated_img, cv2.COLOR_BGR2RGB)
    
    plt.figure(figsize=(10, 8))
    plt.imshow(annotated_img_rgb)
    plt.axis('off')
    plt.title(f"YOLOv12 Detection - {test_img.name}")
    plt.show()


## Step 6: Test Model on Sample Image


# Suppress verbose logs

import logging
logging.getLogger("ultralytics").setLevel(logging.ERROR)
